In [ ]:
import json, requests
import html5lib
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt


In [ ]:
def scrape_part(web_page,class_title):
  result=''
  page = requests.get(web_page)
  soup = BeautifulSoup(page.content, 'html.parser')


  # Convert the list element to a string
  div_element=soup.find_all('div', class_=class_title)
  html = str(div_element)

  # Use regular expressions to find headers and their content
  pattern = r'<h\d>(.*?)<\/h\d>(.*?)((?=<h\d>)|$)'
  matches = re.findall(pattern, html, re.DOTALL)

  # Extract headers and content
  headers = []
  content = []

  for match in matches:
    headers.append(match[0])
    content.append(match[1])

  # Print the headers and their corresponding content
  for i in range(len(set(headers))):
    soup2 = BeautifulSoup(content[i], 'html.parser')
    result+=headers[i]+':'+'\n'+soup2.get_text()+'\n'
  return result

In [148]:
def scrape(link,filename):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')

    intro_text = ""
    for div in soup.find_all('div', class_='field field--name-field-intro-section-one'):
        intro_text += div.text
        break


    program_highlight =""


    program_highlight =scrape_part(link,'field field--name-field-intro-section-two')

    if program_highlight=='':
      for div in soup.find_all('div', class_='field field--name-field-intro-section-two'):
        program_highlight += div.text
        break
      # Find the div containing the program highlights
      div_element = soup.find('div', class_="field--name-field-bullet-row")

      # Find the header
      header = div_element.find_previous_sibling('h3')

      # Append the header text to the result
      program_highlight +='\n'+ header.get_text() + ':\n\n'

      # Find all the <p> tags within the div
      paragraphs = div_element.find_all('p')

      # Extract the text content of each <p> tag and append to the result
      for p in paragraphs:
        program_highlight += p.get_text() + '\n'

    rest_text  =scrape_part(link,'field field--name-field-kicker-body')

    if rest_text=='':
      for div in soup.find_all('div', class_='field field--name-field-kicker-body'):
        rest_text += div.text
        break



    with open(filename, 'w') as f:
        f.write(intro_text)
        f.write("\n")
        f.write(program_highlight)
        f.write("\n")
        f.write(rest_text)

    return

In [149]:
pages=["https://www.yu.edu/katz/ai",'https://www.yu.edu/katz/biotech','https://www.yu.edu/katz/cyber',
       'https://www.yu.edu/katz/data-analytics','https://www.yu.edu/katz/digital-marketing-media','https://www.yu.edu/katz/slp',
       'https://www.yu.edu/katz/physics','https://www.yu.edu/katz/physician-assistant','https://www.yu.edu/katz/occupational-therapy',
       'https://www.yu.edu/katz/math-phd','https://www.yu.edu/katz/math-ma',]


In [150]:
for link in pages:

    scrape(link,link[link.rfind('/')+1:]+'.txt')